# 1. Import dependancies 

In [1]:
# Import Gym dependancies
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

# Import helpers
import numpy as np 
import os 
import random 

# Import stable baselines dependancies
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(


# 2. Types of spaces

### There are four key types of spaces:
* Box
* Discrete
* MultiBinary
* MultiDiscrete

### and two wrapper spaces which help group the other spaces together:
* Tuple
* Dict

In [2]:
Discrete(3).sample()

2

In [3]:
Box(0,10,shape=(3,3)).sample()

array([[5.698666 , 4.3980002, 1.4169606],
       [2.5100489, 1.3895131, 3.0238829],
       [4.166131 , 4.944981 , 5.1853814]], dtype=float32)

In [4]:
Tuple((Discrete(3), Box(0,1,shape= (3,3)))).sample()

(0,
 array([[0.84023345, 0.82398653, 0.3969201 ],
        [0.42762095, 0.28051665, 0.6627267 ],
        [0.2700993 , 0.1177856 , 0.28714997]], dtype=float32))

In [5]:
Dict({'Height': Discrete(2),
      'Speed': Box(0,100,shape=(1,))}).sample()

OrderedDict([('Height', 1), ('Speed', array([16.107346], dtype=float32))])

In [6]:
MultiBinary(4).sample()

array([1, 1, 1, 1], dtype=int8)

In [7]:
MultiDiscrete([10,20,30,40]).sample()

array([0, 4, 0, 0])

# 3. Building an environment
- Build an agent to give the best shower possible
- Random temperature due to other people in the building
- Want the temperature between 37 and 39 degrees, the agent will have to learn this 

In [8]:
class ShowerEnv(Env):
    def __init__(self):
        self.action_space = Discrete(3) # 3 actions (Up, Down, Nothing)
        self.observation_space = Box(low = 0, high = 100, shape = (1,)) 
        self.state = 38 + random.randint(-3,3)
        self.shower_length = 60 # 60 second shower, fast shower!
        
    def step(self, action):
        # Apply temperature adjustment
        self.state += action -1
        
        # Need to decrease shower length by 1 for every action that is taken
        self.shower_length -=1
        
        # Calculate reward
        if self.state >= 37 and self.state <=39:
            reward = 1
        else:
            reward = -1
        
        if self.shower_length <= 0 :
            done = True
        else:
            done = False
            
        info = {}
        
        return self.state, reward, done, info
    
    def render(self):
        # Not implementing visibility but could with viz
        pass
        
    def reset(self):
        self.state = np.array([38 + random.randint(-3,3)]).astype(float)
        self.shower_length = 60 
        return self.state
        

In [9]:
env = ShowerEnv()

In [10]:
env.observation_space.sample()

array([21.777327], dtype=float32)

In [11]:
env.action_space.sample()

0

# 4. Test environment

In [12]:
episodes = 5

for episode in range(1, episodes +1):
    obs = env.reset()
    score = 0
    done = False
     
    while not done:
        action = env.action_space.sample()
        obs, reward, done, info = env.step(action)
        score += reward
        
    print(f'Episode: {episode} Score:{score}')
    

Episode: 1 Score:-40
Episode: 2 Score:-16
Episode: 3 Score:-60
Episode: 4 Score:-30
Episode: 5 Score:16


# 5. Train model 

In [13]:
log_path = os.path.join('Training', 'Logs')
model = PPO('MlpPolicy', env, verbose = 1, tensorboard_log = log_path)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [14]:
model.learn(total_timesteps = 100000)

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2
Logging to Training/Logs/PPO_18
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 60       |
|    ep_rew_mean     | -30.8    |
| time/              |          |
|    fps             | 346      |
|    iterations      | 1        |
|    time_elapsed    | 5        |
|    total_timesteps | 2048     |
---------------------------------
-------------------------------------------
| rollout/                |               |
|    ep_len_mean          | 60            |
|    ep_rew_mean          | -28.6         |
| time/                   |               |
|    fps                  | 558           |
|    iterations           | 2             |
|    time_elapsed         | 7             |
|    total_timesteps      | 4096          |
| train/                 

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | -19.8       |
| time/                   |             |
|    fps                  | 1058        |
|    iterations           | 11          |
|    time_elapsed         | 21          |
|    total_timesteps      | 22528       |
| train/                  |             |
|    approx_kl            | 0.012629129 |
|    clip_fraction        | 0.0705      |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.07       |
|    explained_variance   | -6.32e-06   |
|    learning_rate        | 0.0003      |
|    loss                 | 30.8        |
|    n_updates            | 100         |
|    policy_gradient_loss | -0.0037     |
|    value_loss           | 62.4        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60           |
|    ep_rew_mean          | 3.86         |
| time/                   |              |
|    fps                  | 1212         |
|    iterations           | 21           |
|    time_elapsed         | 35           |
|    total_timesteps      | 43008        |
| train/                  |              |
|    approx_kl            | 0.0046374584 |
|    clip_fraction        | 0.101        |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.05        |
|    explained_variance   | 0.00211      |
|    learning_rate        | 0.0003       |
|    loss                 | 15.2         |
|    n_updates            | 200          |
|    policy_gradient_loss | -0.00563     |
|    value_loss           | 37.2         |
------------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_m

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 38.9        |
| time/                   |             |
|    fps                  | 1280        |
|    iterations           | 31          |
|    time_elapsed         | 49          |
|    total_timesteps      | 63488       |
| train/                  |             |
|    approx_kl            | 0.014318554 |
|    clip_fraction        | 0.135       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.981      |
|    explained_variance   | 4.59e-05    |
|    learning_rate        | 0.0003      |
|    loss                 | 29          |
|    n_updates            | 300         |
|    policy_gradient_loss | 0.00347     |
|    value_loss           | 54.3        |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60    

-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 60          |
|    ep_rew_mean          | 49.9        |
| time/                   |             |
|    fps                  | 1315        |
|    iterations           | 41          |
|    time_elapsed         | 63          |
|    total_timesteps      | 83968       |
| train/                  |             |
|    approx_kl            | 0.008248526 |
|    clip_fraction        | 0.164       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.875      |
|    explained_variance   | 2.8e-06     |
|    learning_rate        | 0.0003      |
|    loss                 | 46.8        |
|    n_updates            | 400         |
|    policy_gradient_loss | 0.00576     |
|    value_loss           | 88.1        |
-----------------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 60  

# 6. Save model 

In [15]:
shower_path = os.path.join('Training/Saved Models', 'Shower_Model_PPO')

In [16]:
model.save(shower_path)

In [17]:
del model

In [18]:
model = PPO.load(shower_path)

In [19]:
env.reset()

array([37.])

In [20]:
evaluate_policy(model, env, n_eval_episodes=10)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(59.2, 0.9797958971132712)